In [1]:
from pymongo import MongoClient
import datetime
import pandas as pd
import numpy as np
import itertools
from collections import Counter

client = MongoClient("localhost", 27017)
db = client["reddit_polarization"]

In [64]:
# http://www.ranks.nl/stopwords
# MYSQL Stopwords
sw = ["a's", 'accordingly', 'again', 'allows', 'also', 'amongst',
      'anybody', 'anyways', 'appropriate', 'aside', 'available',
      'because', 'before', 'below', 'between', 'by', "can't",
      'certain', 'com', 'consider', 'corresponding', 'definitely',
      'different', "don't", 'each', 'else', 'et', 'everybody',
      'exactly', 'fifth', 'follows', 'four', 'gets', 'goes',
      'greetings', 'has', 'he', 'her', 'herein', 'him', 'how',
      "i'm", 'immediate', 'indicate', 'instead', 'it', 'itself',
      'know', 'later', 'lest', 'likely', 'ltd', 'me', 'more',
      'must', 'nd', 'needs', 'next', 'none', 'nothing', 'of',
      'okay', 'ones', 'others', 'ourselves', 'own', 'placed',
      'probably', 'rather', 'regarding', 'right', 'saying',
      'seeing', 'seen', 'serious', 'she', 'so', 'something',
      'soon', 'still', "t's", 'th', 'that', 'theirs', 'there',
      'therein', "they'd", 'third', 'though', 'thus', 'toward',
      'try', 'under', 'unto', 'used', 'value', 'vs', 'way',
      "we've", "weren't", 'whence', 'whereas', 'whether', "who's",
      'why', 'within', "wouldn't", "you'll", 'able', 'across',
      'against', 'almost', 'although', 'an', 'anyhow', 'anywhere',
      'are', 'ask', 'away', 'become', 'beforehand', 'beside',
      'beyond', "c'mon", 'cannot', 'certainly', 'come',
      'considering', 'could', 'described', 'do', 'done', 'edu',
      'elsewhere', 'etc', 'everyone', 'example', 'first', 'for',
      'from', 'getting', 'going', 'had', "hasn't", "he's", 'here',
      'hereupon', 'himself', 'howbeit', "i've", 'in', 'indicated',
      'into', "it'd", 'just', 'known', 'latter', 'let', 'little',
      'mainly', 'mean', 'moreover', 'my', 'near', 'neither', 'nine',
      'noone', 'novel', 'off', 'old', 'only', 'otherwise', 'out',
      'particular', 'please', 'provides', 'rd', 'regardless',
      'said', 'says', 'seem', 'self', 'seriously', 'should', 'some',
      'sometime', 'sorry', 'sub', 'take', 'than', "that's", 'them',
      "there's", 'theres', "they'll", 'this', 'three', 'to',
      'towards', 'trying', 'unfortunately', 'up', 'useful',
      'various', 'want', 'we', 'welcome', 'what', 'whenever',
      'whereby', 'which', 'whoever', 'will', 'without', 'yes',
      "you're", 'about', 'actually', "ain't", 'alone', 'always',
      'and', 'anyone', 'apart', "aren't", 'asking', 'awfully',
      'becomes', 'behind', 'besides', 'both', "c's", 'cant',
      'changes', 'comes', 'contain', "couldn't", 'despite', 'does',
      'down', 'eg', 'enough', 'even', 'everything', 'except', 'five',
      'former', 'further', 'given', 'gone', "hadn't", 'have', 'hello',
      "here's", 'hers', 'his', 'however', 'ie', 'inasmuch',
      'indicates', 'inward', "it'll", 'keep', 'knows', 'latterly',
      "let's", 'look', 'many', 'meanwhile', 'most', 'myself', 'nearly',
      'never', 'no', 'nor', 'now', 'often', 'on', 'onto', 'ought',
      'outside', 'particularly', 'plus', 'que', 're', 'regards', 'same',
      'second', 'seemed', 'selves', 'seven', "shouldn't", 'somebody',
      'sometimes', 'specified', 'such', 'taken', 'thank', 'thats',
      'themselves', 'thereafter', 'thereupon', "they're", 'thorough',
      'through', 'together', 'tried', 'twice', 'unless', 'upon', 'uses',
      'very', 'wants', "we'd", 'well', "what's", 'where', 'wherein',
      'while', 'whole', 'willing', "won't", 'yet', "you've", 'above',
      'after', 'all', 'along', 'am', 'another', 'anything', 'appear',
      'around', 'associated', 'be', 'becoming', 'being', 'best', 'brief',
      'came', 'cause', 'clearly', 'concerning', 'containing', 'course',
      'did', "doesn't", 'downwards', 'eight', 'entirely', 'ever',
      'everywhere', 'far', 'followed', 'formerly', 'furthermore', 'gives',
      'got', 'happens', "haven't", 'help', 'hereafter', 'herself', 'hither',
      "i'd", 'if', 'inc', 'inner', 'is', "it's", 'keeps', 'last', 'least',
      'like', 'looking', 'may', 'merely', 'mostly', 'name', 'necessary',
      'nevertheless', 'nobody', 'normally', 'nowhere', 'oh', 'once', 'or',
      'our', 'over', 'per', 'possible', 'quite', 'really', 'relatively',
      'saw', 'secondly', 'seeming', 'sensible', 'several', 'since',
      'somehow', 'somewhat', 'specify', 'sup', 'tell', 'thanks', 'the',
      'then', 'thereby', 'these', "they've", 'thoroughly', 'throughout',
      'too', 'tries', 'two', 'unlikely', 'us', 'using', 'via', 'was', "we'll",
      'went', 'whatever', "where's", 'whereupon', 'whither', 'whom', 'wish',
      'wonder', 'you', 'your', 'according', 'afterwards', 'allow', 'already',
      'among', 'any', 'anyway', 'appreciate', 'as', 'at', 'became', 'been',
      'believe', 'better', 'but', 'can', 'causes', 'co', 'consequently',
      'contains', 'currently', "didn't", 'doing', 'during', 'either',
      'especially', 'every', 'ex', 'few', 'following', 'forth', 'get', 'go',
      'gotten', 'hardly', 'having', 'hence', 'hereby', 'hi', 'hopefully',
      "i'll", 'ignored', 'indeed', 'insofar', "isn't", 'its', 'kept', 'lately',
      'less', 'liked', 'looks', 'maybe', 'might', 'much', 'namely', 'need',
      'new', 'non', 'not', 'obviously', 'ok', 'one', 'other', 'ours', 'overall',
      'perhaps', 'presumably', 'qv', 'reasonably', 'respectively', 'say', 'see',
      'seems', 'sent', 'shall', 'six', 'someone', 'somewhere', 'specifying',
      'sure', 'tends', 'thanx', 'their', 'thence', 'therefore', 'they', 'think',
      'those', 'thru', 'took', 'truly', 'un', 'until', 'use', 'usually', 'viz',
      "wasn't", "we're", 'were', 'when', 'whereafter', 'wherever', 'who',
      'whose', 'with', 'would', "you'd", 'yours', 'yourself', 'yourselves', 'zero',
      'guy', 'day', 'great', 'thing', 'fact', 'point', 'lot']
sw = set(sw)

In [3]:
men = pd.DataFrame(list(db["MensRights"].find({}, {"author": 1, "score": 1, "subreddit": 1, "_id": 0})))
fem = pd.DataFrame(list(db["Feminism"].find({}, {"author": 1, "score": 1, "subreddit": 1, "_id": 0})))
print men.shape, fem.shape

In [6]:
authors = pd.concat([men, fem], axis=0)

In [7]:
def agg_author(df):
    men_score = df[df["subreddit"] == "MensRights"]["score"].mean()
    men_count = df[df["subreddit"] == "MensRights"]["score"].shape[0]
    fem_score = df[df["subreddit"] == "Feminism"]["score"].mean()
    fem_count = df[df["subreddit"] == "Feminism"]["score"].shape[0]
    return pd.Series([men_score, men_count, fem_score, fem_count], index=["men_score", "men_count", "fem_score", "fem_count"])

In [8]:
author_profile = authors.groupby("author").apply(agg_author)

In [23]:
men_score = author_profile["men_score"][author_profile["men_score"].notnull()]
print men_score.mean(), men_score.min(), men_score.max(), men_score.std()

(4.541925650216907, -266.0, 838.0, 13.85009556426775)

In [24]:
fem_score = author_profile["fem_score"][author_profile["fem_score"].notnull()]
print fem_score.mean(), fem_score.min(), fem_score.max(), fem_score.std()

(4.045237320018844, -94.0, 281.0, 7.291276710126762)

In [47]:
men_author_profile = author_profile[author_profile["men_score"].notnull()][["men_score", "men_count"]]
men_author_profile = men_author_profile.reset_index()

In [48]:
fem_author_profile = author_profile[author_profile["fem_score"].notnull()][["fem_score", "fem_count"]]
fem_author_profile = fem_author_profile.reset_index()

In [62]:
men = pd.DataFrame(list(db["MensRights"].find({}, {"score": 1, "tokens_njv_pos": 1, "_id": 0})))
fem = pd.DataFrame(list(db["Feminism"].find({}, {"score": 1, "tokens_njv_pos": 1, "_id": 0})))
print men.shape, fem.shape

In [67]:
men["rank"] = men["score"].rank(ascending=False)
fem["rank"] = fem["score"].rank(ascending=False)

In [73]:
def pos_filter(x, keep=set("NJ")):
    if len(x) == 0:
        return ""
    
    tokens_pos = x.split(" ")
    tokens_pos = map(lambda pair: pair.split("/"), tokens_pos)
    
    tokens_pos = filter(lambda pair: pair[1] in keep, tokens_pos)

    if len(tokens_pos) == 0:
        return ""
    
    return " ".join(zip(*tokens_pos)[0])

In [75]:
men_x_corpus = men[men["rank"] <= 1000]["tokens_njv_pos"].map(pos_filter)

In [77]:
men_x_corpus.iloc[0]

u'child stranger infant daughter men room mall bum stall team mall security report suspicious male small child bathroom culture much stranger danger penis danger'